In [1]:
import os
import json
import requests
from bs4 import BeautifulSoup
from copy import deepcopy

In [2]:
!ls

brin.html		    hinton.html  world_cat_mapping.json
fetch_page_world_cat.ipynb  sample.html  world_done.json


In [3]:
file_name="world_cat_mapping.json"
with open(file_name,'r') as fd:
    in_df=json.load(fd)

In [4]:
def fetch_url(p_id):
    base_url=f"http://www.worldcat.org/identities/{p_id}"
    return base_url

In [5]:
def strip_spaces(curr_text):
    return ' '.join(curr_text.split())

In [6]:
#in_df=in_df[:10]

In [7]:
len(in_df)

3254

In [8]:
final_df=[]

In [9]:
def save_results():
    with open("./world_done.json",'w') as fd:
        json.dump({"done_cnt":done_cnt,'result':final_df, "failures":unlucky_list},fd,indent=3)

In [10]:
def fetch_content_from_tr(tr_tag):
    head_text=tr_tag.find("th").text
    td_tag=tr_tag.find("td")
    #print("td tag is ", td_tag)
    insider_tags=td_tag.find_all()
    if len(insider_tags)==0:
        return head_text,strip_spaces(td_tag.text)
    return head_text,[strip_spaces(x.text) for x in td_tag.find_all(recursive=False)]

In [11]:
def fetch_overview(main_tag):
    table_tag = main_tag.find("table", {"id": "identitiestbl_overview"})
    ans_dict={}
    if table_tag==None:
        return ans_dict
    #print("Table tag is ", table_tag)
    tr_tags=table_tag.find_all("tr")
    for curr_tr_tag in tr_tags:
        key, val=fetch_content_from_tr(curr_tr_tag)
        ans_dict[key]=val
    
    return ans_dict
    


In [12]:
def fetch_related_entities(main_tag):
    div_tag = main_tag.find("div", {"id": "identitiesassociates"})
    #print(div_tag)
    ans_arr=[]
    if div_tag==None:
        return ans_arr
    #print("Table tag is ", table_tag)
    li_tags=div_tag.find("ul").find_all("li")
    for curr_li_tag in li_tags:
        this_text=curr_li_tag.find("a").text
        ans_arr.append(this_text)
    
    return ans_arr
    


In [13]:
def fetch_associated_subjects(main_tag):
    div_tag = main_tag.find("div", {"id": "identitiesFASTCloud"})
    #print(div_tag)
    ans_arr=[]
    if div_tag==None:
        return ans_arr
    #print("Table tag is ", table_tag)
    a_tags=div_tag.find_all("a")
    ans_arr=[x.text for x in a_tags]
    
    return ans_arr
    


In [14]:
import re
def fetch_books_about(main_tag):
    div_tag = main_tag.find("div", {"id":"identitiesworksabout"})
    ans_arr=[]
    if div_tag==None:
        return ans_arr
    #print("Table tag is ", table_tag)
    li_tags=div_tag.find("ul").find_all("li")
    for curr_li_tag in li_tags:
        this_text=curr_li_tag.text
        this_text=re.sub(r'<.*>',"",this_text)
        ans_arr.append(this_text)
    ans_arr=list(map(strip_spaces, ans_arr))
    return ans_arr  

In [15]:
def fetch_books_by(main_tag):
    div_tags = main_tag.find_all("div", {"class":"identitiesWorksByEntry"})
    #print(main_tag)
    #print(div_tags)
    ans_arr=[]
    if div_tags==[]:
        return ans_arr
    #print("Table tag is ", table_tag)
    for curr_div_tag in div_tags:
        this_text=curr_div_tag.text
        this_text=re.sub(r'<br>',". ",this_text)
        this_text=re.sub(r'<.*>',"",this_text)
        ans_arr.append(this_text)
    ans_arr=list(map(strip_spaces, ans_arr))
    return ans_arr  

In [16]:
from copy import deepcopy

In [17]:
unlucky_list=[]
done_cnt=0
for idx, person_name in enumerate(in_df):
    #print(person_name)
    #print(in_df[person_name])
    if in_df[person_name]==None:
        unlucky_list.append((person_name,"ID not found in Wikipedia"))
        continue
    try:
        # id exists in wikipedia
        curr_link=fetch_url(in_df[person_name])
        page = requests.get(curr_link)
        soup = BeautifulSoup(page.text, 'html.parser')
        master_dict={}
        master_dict['person_name']=person_name
        master_dict['overwiew_section']=fetch_overview(soup)
        master_dict['works_about_person']=fetch_books_about(soup)
        master_dict['works_by_person']=fetch_books_by(soup)
        master_dict['related_entities']=fetch_related_entities(soup)
        master_dict['associated_subjects']=fetch_associated_subjects(soup)
        final_df.append(deepcopy(master_dict))
        done_cnt+=1
    except Exception as E:
        unlucky_list.append((person_name,E))
        
    if done_cnt%100==0:
        save_results()
        #print(done_cnt)
        #break

In [18]:
save_results()

In [19]:
unlucky_list

[('Karen Myers', 'ID not found in Wikipedia'),
 ('Raymond J. Mooney', 'ID not found in Wikipedia'),
 ('Stathis Zachos', 'ID not found in Wikipedia'),
 ('Jay Freeman', 'ID not found in Wikipedia'),
 ('Hui Lei', 'ID not found in Wikipedia'),
 ('Sandy Lerner', 'ID not found in Wikipedia'),
 ('Joseph G. Davis', 'ID not found in Wikipedia'),
 ('Neil Wiseman', 'ID not found in Wikipedia'),
 ('Jim Gray', 'ID not found in Wikipedia'),
 ('Gretchen Matthews', 'ID not found in Wikipedia'),
 ('Burt Kaliski', 'ID not found in Wikipedia'),
 ('Ira Fuchs', 'ID not found in Wikipedia'),
 ('Prasad V. Tetali', 'ID not found in Wikipedia'),
 ('Elaine Cohen', 'ID not found in Wikipedia'),
 ('Vijaykrishnan Narayanan', 'ID not found in Wikipedia'),
 ('Timo Honkela', 'ID not found in Wikipedia'),
 ('Melanie Rieback', 'ID not found in Wikipedia'),
 ('Harry Mairson', 'ID not found in Wikipedia'),
 ('Thomas Sederberg', 'ID not found in Wikipedia'),
 ('Jeanne Clare Adams', 'ID not found in Wikipedia'),
 ('Nina Am